In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

# ! pip install --upgrade openpyxl
# ! pip uninstall openpyxl
# ! pip install openpyxl

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
# plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['font.family'] ='AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

## hw_8_2

In [3]:
# 1. 데이터 불러오기
# pandas의 read_excel을 사용하여 엑셀 파일에서 데이터를 불러옵니다.
# file_path는 불러올 엑셀 파일의 경로입니다.
# sheet_name은 불러올 시트의 이름을 지정하는데, 여기서는 '2023년 01월' 데이터를 가져옵니다.
# 참고: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html
file_path = 'data/2023년 01월 서울시 교통량.xlsx'
data = pd.read_excel(file_path, sheet_name='2023년 01월')
print(data.info())
data.head()
# 데이터를 살펴보면 현재 일자와 요일이 일치하지 않습니다. 그렇기 때문에 일자 칼럼을 이용하여 요일을 다시 계산해야 합니다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8618 entries, 0 to 8617
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   일자      8618 non-null   int64  
 1   요일      8618 non-null   object 
 2   지점명     8618 non-null   object 
 3   지점번호    8618 non-null   object 
 4   방향      8618 non-null   object 
 5   구분      8618 non-null   object 
 6   0시      6589 non-null   float64
 7   1시      6588 non-null   float64
 8   2시      6584 non-null   float64
 9   3시      6569 non-null   float64
 10  4시      6567 non-null   float64
 11  5시      6566 non-null   float64
 12  6시      6569 non-null   float64
 13  7시      6570 non-null   float64
 14  8시      6572 non-null   float64
 15  9시      6576 non-null   float64
 16  10시     6581 non-null   float64
 17  11시     6588 non-null   float64
 18  12시     6587 non-null   float64
 19  13시     6587 non-null   float64
 20  14시     6592 non-null   float64
 21  15시     6592 non-null   float64
 22  

,일자,요일,지점명,지점번호,방향,구분,0시,1시,2시,3시,...,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시
0,20230101,일,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,564.0,414.0,268.0,208.0,...,1388.0,1370.0,1322.0,1263.0,979.0,875.0,785.0,824.0,552.0,421.0
1,20230102,월,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,294.0,190.0,146.0,134.0,...,1459.0,1571.0,1566.0,1792.0,1582.0,1248.0,997.0,973.0,872.0,616.0
2,20230103,화,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,419.0,271.0,203.0,177.0,...,1663.0,1678.0,1711.0,1851.0,1697.0,1265.0,1081.0,1090.0,883.0,757.0
3,20230104,수,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,502.0,335.0,247.0,192.0,...,1679.0,1798.0,1719.0,1807.0,1679.0,1240.0,1112.0,1069.0,909.0,769.0
4,20230105,목,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,523.0,334.0,235.0,196.0,...,1530.0,1627.0,1805.0,1798.0,1770.0,1289.0,1152.0,1111.0,1030.0,756.0


In [4]:
# 2. '일자' 컬럼을 datetime 형식으로 변환
# 이를 datetime 형식으로 변환하면 날짜 관련 작업이 더 쉬워집니다.
# pandas의 to_datetime 함수를 사용하여 '일자' 데이터를 날짜 형식으로 변환합니다.
# (참고: '일자' 컬럼은 날짜를 나타내는 값인데, 이 컬럼의 각 값들은 정수 형식(YYYYMMDD)으로 되어 있습니다.)
# 참고: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html
data['일자'] = pd.to_datetime(data['일자'], format='%Y%m%d')

# 3. 날짜를 기반으로 요일을 자동으로 계산
# 요일이 정수로 변환되면 학습 모델에서 이를 숫자형 데이터로 처리할 수 있습니다.
# datetime 형식으로 변환된 '일자' 컬럼을 기반으로 요일을 계산합니다.
# dt.weekday는 '월요일'=0, '일요일'=6을 반환하므로, +1을 해주어 '월요일'=1, '일요일'=7로 변환합니다.
# 참고: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html
data['요일'] =data['일자'].dt.weekday + 1

# 4. 데이터 필터링
# '성산로(금화터널)' 지점에서 '유입' 방향으로 들어오는 데이터만을 선택합니다.
# 데이터를 먼저 필터링하여 분석에 필요한 데이터만 남깁니다.
# Boolean indexing을 사용하여 조건에 맞는 데이터만 필터링할 수 있습니다.
# 예: filtered_data = 데이터프레임[데이터프레임['컬럼명'] == '값' & 데이터프레임['컬럼명'] == '값']
# 참고: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#boolean-indexing
filtered_data = data[(data['지점명']=='성산로(금화터널)') & (data['방향']=='유입')]
print(filtered_data.info())
filtered_data.head()


<class 'pandas.core.frame.DataFrame'>
Index: 31 entries, 0 to 30
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일자      31 non-null     datetime64[ns]
 1   요일      31 non-null     int32         
 2   지점명     31 non-null     object        
 3   지점번호    31 non-null     object        
 4   방향      31 non-null     object        
 5   구분      31 non-null     object        
 6   0시      31 non-null     float64       
 7   1시      31 non-null     float64       
 8   2시      31 non-null     float64       
 9   3시      31 non-null     float64       
 10  4시      31 non-null     float64       
 11  5시      31 non-null     float64       
 12  6시      31 non-null     float64       
 13  7시      31 non-null     float64       
 14  8시      31 non-null     float64       
 15  9시      31 non-null     float64       
 16  10시     31 non-null     float64       
 17  11시     31 non-null     float64       
 18  12시     31 non-nu

,일자,요일,지점명,지점번호,방향,구분,0시,1시,2시,3시,...,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시
0,2023-01-01,7,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,564.0,414.0,268.0,208.0,...,1388.0,1370.0,1322.0,1263.0,979.0,875.0,785.0,824.0,552.0,421.0
1,2023-01-02,1,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,294.0,190.0,146.0,134.0,...,1459.0,1571.0,1566.0,1792.0,1582.0,1248.0,997.0,973.0,872.0,616.0
2,2023-01-03,2,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,419.0,271.0,203.0,177.0,...,1663.0,1678.0,1711.0,1851.0,1697.0,1265.0,1081.0,1090.0,883.0,757.0
3,2023-01-04,3,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,502.0,335.0,247.0,192.0,...,1679.0,1798.0,1719.0,1807.0,1679.0,1240.0,1112.0,1069.0,909.0,769.0
4,2023-01-05,4,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,523.0,334.0,235.0,196.0,...,1530.0,1627.0,1805.0,1798.0,1770.0,1289.0,1152.0,1111.0,1030.0,756.0


In [5]:
# 5. 피처와 타깃 값 설정
# 독립변수는 2차원, 종속 변수는 1차원의 값을 입력해야 한다.
# '요일'을 독립 변수로, '0시' 교통량을 종속 변수로 설정합니다.
# 참고: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html
X = filtered_data[['요일']]
y = filtered_data['0시']

# 6. 학습 데이터와 테스트 데이터 분할
# 학습 데이터와 테스트 데이터를 나누는 함수로 train_test_split을 사용합니다.
# 참고: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 7. 선형 회귀 모델 정의 및 학습
# scikit-learn의 LinearRegression 모델을 사용하여 학습 데이터를 학습시킵니다.
# 참고: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
model = LinearRegression()
model.fit(X_train, y_train)

# 8. 모델 평가
# 학습된 모델을 평가하기 위해 R² 점수를 사용합니다. 1에 가까울수록 좋은 성능을 의미합니다.
# scikit-learn의 score() 함수는 기본적으로 R² 점수를 반환합니다.
# 참고: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.score
r2_score = model.score(X_test, y_test)
print(f"R²: {r2_score}")

R²: 0.2835616050573251


## hw_8_4

In [5]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# 1. 데이터 불러오기
# pandas의 read_excel을 사용하여 엑셀 파일에서 데이터를 불러옵니다.
# file_path는 불러올 엑셀 파일의 경로입니다.
# sheet_name은 불러올 시트의 이름을 지정합니다. 여기서는 '2023년 01월' 데이터를 가져옵니다.
# 참고: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html
file_path = 'data/2023년 01월 서울시 교통량.xlsx'
data = pd.read_excel(file_path, sheet_name='2023년 01월')
print(data.info())
data.head()
# 데이터를 살펴보면 현재 일자와 요일이 일치하지 않습니다. 그렇기 때문에 일자 칼럼을 이용하여 요일을 다시 계산해야 합니다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8618 entries, 0 to 8617
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   일자      8618 non-null   int64  
 1   요일      8618 non-null   object 
 2   지점명     8618 non-null   object 
 3   지점번호    8618 non-null   object 
 4   방향      8618 non-null   object 
 5   구분      8618 non-null   object 
 6   0시      6589 non-null   float64
 7   1시      6588 non-null   float64
 8   2시      6584 non-null   float64
 9   3시      6569 non-null   float64
 10  4시      6567 non-null   float64
 11  5시      6566 non-null   float64
 12  6시      6569 non-null   float64
 13  7시      6570 non-null   float64
 14  8시      6572 non-null   float64
 15  9시      6576 non-null   float64
 16  10시     6581 non-null   float64
 17  11시     6588 non-null   float64
 18  12시     6587 non-null   float64
 19  13시     6587 non-null   float64
 20  14시     6592 non-null   float64
 21  15시     6592 non-null   float64
 22  

,일자,요일,지점명,지점번호,방향,구분,0시,1시,2시,3시,...,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시
0,20230101,일,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,564.0,414.0,268.0,208.0,...,1388.0,1370.0,1322.0,1263.0,979.0,875.0,785.0,824.0,552.0,421.0
1,20230102,월,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,294.0,190.0,146.0,134.0,...,1459.0,1571.0,1566.0,1792.0,1582.0,1248.0,997.0,973.0,872.0,616.0
2,20230103,화,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,419.0,271.0,203.0,177.0,...,1663.0,1678.0,1711.0,1851.0,1697.0,1265.0,1081.0,1090.0,883.0,757.0
3,20230104,수,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,502.0,335.0,247.0,192.0,...,1679.0,1798.0,1719.0,1807.0,1679.0,1240.0,1112.0,1069.0,909.0,769.0
4,20230105,목,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,523.0,334.0,235.0,196.0,...,1530.0,1627.0,1805.0,1798.0,1770.0,1289.0,1152.0,1111.0,1030.0,756.0


In [6]:
# 2. '일자' 컬럼을 datetime 형식으로 변환
# 이를 datetime 형식으로 변환하면 날짜 관련 작업이 더 쉬워집니다.
# pandas의 to_datetime 함수를 사용하여 '일자' 데이터를 날짜 형식으로 변환합니다.
# (참고: '일자' 컬럼은 날짜를 나타내는 값인데, 이 컬럼의 각 값들은 정수 형식(YYYYMMDD)으로 되어 있습니다.)
# 참고: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html
data['일자'] = pd.to_datetime(data['일자'], format='%Y%m%d')

# 3. 날짜를 기반으로 요일을 자동으로 계산
# 요일 정보는 모델에서 숫자형 데이터로 처리할 수 있도록 변환되어야 합니다.
# datetime 형식으로 변환된 '일자' 컬럼을 기반으로 요일을 계산합니다.
# dt.weekday는 '월요일'=0, '일요일'=6을 반환하므로, +1을 해주어 '월요일'=1, '일요일'=7로 변환합니다.
# 참고: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html
data['요일'] = data['일자'].dt.weekday + 1

# 4. 시간대 변수를 그대로 사용
# 기존 데이터셋에서 '0시'부터 '23시'까지 시간대별 교통량 데이터가 이미 존재합니다.
# 이를 독립 변수로 사용하여 각 시간대의 교통량 정보를 피처로 추가합니다.
# 시간대 컬럼은 '0시', '1시'... '23시' 등의 이름으로 존재하므로 그대로 사용할 수 있습니다.

# 5. 데이터 필터링
# '성산로(금화터널)' 지점에서 '유입' 방향으로 들어오는 데이터만 필터링합니다.
# Boolean indexing을 사용하여 조건을 만족하는 행만 선택합니다.
# Boolean indexing: 조건이 True인 데이터만 선택합니다.
# 예: filtered_data = 데이터프레임[(데이터프레임['컬럼명'] == '값') & (데이터프레임['컬럼명'] == '값')]
# 참고: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#boolean-indexing
filtered_data = data[(data['지점명']=='성산로(금화터널)') & (data['방향']=='유입')]
print(filtered_data.info())
filtered_data.head()

<class 'pandas.core.frame.DataFrame'>
Index: 31 entries, 0 to 30
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일자      31 non-null     datetime64[ns]
 1   요일      31 non-null     int32         
 2   지점명     31 non-null     object        
 3   지점번호    31 non-null     object        
 4   방향      31 non-null     object        
 5   구분      31 non-null     object        
 6   0시      31 non-null     float64       
 7   1시      31 non-null     float64       
 8   2시      31 non-null     float64       
 9   3시      31 non-null     float64       
 10  4시      31 non-null     float64       
 11  5시      31 non-null     float64       
 12  6시      31 non-null     float64       
 13  7시      31 non-null     float64       
 14  8시      31 non-null     float64       
 15  9시      31 non-null     float64       
 16  10시     31 non-null     float64       
 17  11시     31 non-null     float64       
 18  12시     31 non-nu

,일자,요일,지점명,지점번호,방향,구분,0시,1시,2시,3시,...,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시
0,2023-01-01,7,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,564.0,414.0,268.0,208.0,...,1388.0,1370.0,1322.0,1263.0,979.0,875.0,785.0,824.0,552.0,421.0
1,2023-01-02,1,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,294.0,190.0,146.0,134.0,...,1459.0,1571.0,1566.0,1792.0,1582.0,1248.0,997.0,973.0,872.0,616.0
2,2023-01-03,2,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,419.0,271.0,203.0,177.0,...,1663.0,1678.0,1711.0,1851.0,1697.0,1265.0,1081.0,1090.0,883.0,757.0
3,2023-01-04,3,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,502.0,335.0,247.0,192.0,...,1679.0,1798.0,1719.0,1807.0,1679.0,1240.0,1112.0,1069.0,909.0,769.0
4,2023-01-05,4,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,523.0,334.0,235.0,196.0,...,1530.0,1627.0,1805.0,1798.0,1770.0,1289.0,1152.0,1111.0,1030.0,756.0


In [8]:
# 6. 피처와 타깃 값 설정
# '요일'과 '시간대' (1시~23시)의 정보를 피처로 사용하고, '0시' 시간대의 교통량을 타깃 값으로 설정합니다.
# 시간대 정보는 이미 데이터셋에서 '1시', '2시' 등의 컬럼으로 존재하므로 이를 독립 변수로 추가합니다.
# pandas의 concat 함수를 사용하여 피처를 결합합니다.
# 참고: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html
# 힌트: 시간대 컬럼은 '1시', '2시', ..., '23시'로 되어 있으며, 이를 for 루프를 통해 추가할 수 있습니다.
# 시간대 컬럼의 이름은 문자열로 처리됩니다. 이를 이용해 여러 열을 동시에 추가할 수 있습니다.
# 예: str(hour) + '시'를 사용하여 1시, 2시 등의 컬럼명을 만들어 보세요.
X = filtered_data[['요일']+[f'{i}시' for i in range(1,24)]]
y = filtered_data['0시']

# 7. 학습 데이터와 테스트 데이터 분할
# 학습 데이터와 테스트 데이터를 나누는 함수로 train_test_split을 사용합니다.
# test_size=0.2는 데이터의 20%를 테스트 데이터로 사용한다는 의미입니다.
# random_state=42는 무작위 분할을 고정하여 재현성을 보장합니다.
# 참고: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 8. 선형 회귀 모델 정의 및 학습
# scikit-learn의 LinearRegression 모델을 사용하여 학습 데이터를 학습시킵니다.
# 모델을 정의한 후, fit 함수를 사용하여 학습 데이터를 학습합니다.
# 참고: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
model = LinearRegression()
model.fit(X_train, y_train)

# 9. 모델 평가
# 학습된 모델을 평가하기 위해 R² 점수를 사용합니다. 1에 가까울수록 성능이 좋은 모델입니다.
# score() 함수는 기본적으로 R² 점수를 반환합니다.
# 참고: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.score
r2_score = model.score(X_test, y_test)
print(f"R²: {r2_score}")


R²: 0.49845696871885603
